In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("fund_sheet.csv")

In [ ]:
fund_categories = df["Sub Category"].unique()
len(fund_categories) 

50

So we are dealing with a total of 50 fund types

Sebi's circulars have a total of 10 (equity) + 16 (debt) + 6 (hybrid) + 2(solution oriented) + 2(Other) + Flexicap = 37 categories

In [7]:
fund_categories

array(['Small Cap Fund', 'Equity Linked Savings Scheme (ELSS)',
       'Mid Cap Fund', 'Flexi Cap Fund', 'Multi Cap Fund',
       'Sectoral Fund - Infrastructure', 'Large & Mid Cap Fund',
       'Sectoral Fund - Technology', 'Value Fund', 'Thematic Fund',
       'Contra Fund', 'Sectoral Fund - Consumption',
       'Sectoral Fund - Energy & Power', 'Multi Asset Allocation Fund',
       'Focused Fund', 'Sectoral Fund - Banks & Financial Services',
       'Aggressive Hybrid Fund', 'Dividend Yield Fund',
       'Sectoral Fund - Pharma & Health Care', 'Large Cap Fund',
       'Sectoral Fund - Service Industry', 'Balanced Advantage Fund',
       'Index Fund', 'Thematic Fund - Global', 'Thematic Fund - MNC',
       'Dynamic Asset Allocation Fund', 'Conservative Hybrid Fund',
       'Equity Savings', 'Medium Duration Fund',
       'Gilt - Short & Mid Term Fund', 'Dynamic Bond Fund',
       'Gilt - Long Term Fund', 'Credit Risk Fund',
       'Medium to Long Duration Fund', 'Short Duration Fund'

Sectoral / Thematic funds are being counted uniquely

In [15]:
count = 1 # 1 count for all sectoral funds
for i in fund_categories:
    if i[:8] == 'Sectoral' or i[:8] == 'Thematic':
        continue
    else:
        count += 1
print(count)

39


The SEBI circular combines Dynamic Asset Allocation and Balanced Advantage so 39 - 1 = 38. Fixed Maturity plan is not present in the circular but is part of https://www.amfiindia.com/investor-corner/knowledge-center/SEBI-categorization-of-mutual-fund-schemes.html